## Initial Setup

1. Download the Ice IX version 1.2.7.0 reference sample from [MalwareBazaar](https://bazaar.abuse.ch/sample/d122c989fe69f0912e7fb2aa838d6eb83995c86b6986f7b9e20fbea287e91ed4/).
2. Unpack the sample with [UNPACME](https://www.unpac.me/results/cefde749-9254-40bd-9a13-363b4c7a3ebe#/).

In [1]:
# Install required Python modules
import sys
!{sys.executable} -m pip install pefile

# Import required Python modules
import hashlib
import pprint
import re
import struct

import pefile

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Load the unpacked sample
fp = open("00224a6627121433bd556bd02592b0b3a82ee49a5f257b6ec23939279a5c12f6.bin", "rb")
sample = fp.read()
fp.close()

pe = pefile.PE(data=sample)

## Decrypt Strings

Pseudocode:

    int __usercall CryptedStrings::_getA@<eax>(unsigned __int16 id@<ax>, int buffer@<edi>)
    {   
      STRINGINFO *s; // eax 
      int i; // ecx 
      int i2; // esi 
      char pb; // dl
      int result; // eax 

      s = &dword_401CD8[2 * id];
      for ( i = 0; i < s->size; *(i2 + buffer) = pb )
      {   
        i2 = i;
        pb = i ^ LOBYTE(s->key) ^ *(s->encodedString + i); 
        ++i;
      }   
      result = s->size;
      *(result + buffer) = 0;
      return result;
    } 

Disassembly:

    .text:00410EE7                               CryptedStrings___getA proc near
    .text:00410EE7
    .text:00410EE7 0F B7 C0                                      movzx   eax, ax
    .text:00410EEA 8D 04 C5 D8 1C 40 00                          lea     eax, dword_401CD8[eax*8]
    .text:00410EF1 33 D2                                         xor     edx, edx
    .text:00410EF3 33 C9                                         xor     ecx, ecx
    .text:00410EF5 66 3B 50 02                                   cmp     dx, [eax+2]
    .text:00410EF9 73 19                                         jnb     short loc_410F14
    .text:00410EFB 56                                            push    esi
    .text:00410EFC
    .text:00410EFC                               loc_410EFC:
    .text:00410EFC 8B 50 04                                      mov     edx, [eax+4]
    .text:00410EFF 0F B7 F1                                      movzx   esi, cx
    .text:00410F02 8A 14 32                                      mov     dl, [edx+esi]
    .text:00410F05 32 10                                         xor     dl, [eax]
    .text:00410F07 32 D1                                         xor     dl, cl
    .text:00410F09 41                                            inc     ecx
    .text:00410F0A 88 14 3E                                      mov     [esi+edi], dl
    .text:00410F0D 66 3B 48 02                                   cmp     cx, [eax+2]
    .text:00410F11 72 E9                                         jb      short loc_410EFC
    .text:00410F13 5E                                            pop     esi
    .text:00410F14
    .text:00410F14                               loc_410F14:
    .text:00410F14 0F B7 40 02                                   movzx   eax, word ptr [eax+2]
    .text:00410F18 C6 04 38 00                                   mov     byte ptr [eax+edi], 0
    .text:00410F1C C3                                            retn
    .text:00410F1C                               CryptedStrings___getA endp

In [3]:
def decrypt_string(pe, id_):
    match = re.search(rb'\x8d\x04\xc5(?P<enc_strings_addr>[\s\S]{4})', 
                      pe.get_memory_mapped_image())
    enc_strings_addr = struct.unpack("I", match.group("enc_strings_addr"))[0]
    
    enc_string_addr = enc_strings_addr + 8 * id_
    enc_string_rva = enc_string_addr - pe.OPTIONAL_HEADER.ImageBase
    
    xor_key = pe.get_word_at_rva(enc_string_rva)
    encbuf_len = pe.get_word_at_rva(enc_string_rva + 2)
    encbuf_addr = pe.get_dword_at_rva(enc_string_rva + 4)
    encbuf_rva = encbuf_addr - pe.OPTIONAL_HEADER.ImageBase
    encbuf = pe.get_data(encbuf_rva, encbuf_len)
    
    plainbuf = b""
    for i in range(len(encbuf)):
        pb = (i ^ xor_key ^ encbuf[i]) & 0xff
        plainbuf += struct.pack("B", pb)
        
    return plainbuf.decode()


for i in range(237):
    plain_string = decrypt_string(pe, i)
    print(f"{i}: {plain_string}")

0: %s://%s:%s@%s/
1: ftp
2: pop3
3: anonymous
4: grabbed\%S_%02u_%02u_%02u.txt
5: Grabbed data from: %s

%S
6: %s%s
Referer: %S
User input: %s
%sPOST data:

%S
7: *EMPTY*
8: *UNKNOWN*
9:  *BLOCKED*
10: Content-Type: %s

11: ZCID: %S

12: application/x-www-form-urlencoded
13: HTTP authentication: username="%s", password="%s"

14: HTTP authentication (encoded): %S

15: Mozilla\Firefox
16: user.js
17: profiles.ini
18: Profile%u
19: IsRelative
20: Path
21: user_pref("network.cookie.cookieBehavior", 0);
user_pref("privacy.clearOnShutdown.cookies", false);
user_pref("security.warn_viewing_mixed", false);
user_pref("security.warn_viewing_mixed.show_once", false);
user_pref("security.warn_submit_insecure", false);
user_pref("security.warn_submit_insecure.show_once", false);

22: user_pref("browser.startup.homepage", "%s");
user_pref("browser.startup.page", 1);

23: Software\Microsoft\Internet Explorer\Main
24: Start Page
25: Software\Microsoft\Internet Explorer\PhishingFilter
26: Enabled
27: E

## Extract Version

Pseudocode:

    ...
    version = 0x1020700;
    v4 = BinStorage::_addItem(binstorage, 4u, 10003, 0x20000, &version);
    ...

Disassembly:

    ...
    .text:004050CB 8D 45 F8                      lea     eax, [ebp+version]
    .text:004050CE 50                            push    eax
    .text:004050CF 57                            push    edi
    .text:004050D0 68 13 27 00 00                push    2713h
    .text:004050D5 6A 04                         push    4
    .text:004050D7 5B                            pop     ebx
    .text:004050D8 8B C6                         mov     eax, esi
    .text:004050DA C7 45 F8 00 07 02 01          mov     [ebp+version], 1020700h
    .text:004050E1 E8 77 4C 01 00                call    BinStorage___addItem
    .text:004050E6 8A D8                         mov     bl, al
    ...

In [4]:
match = re.search(rb'\x68\x13\x27\x00\x00[\s\S]{1,8}?\xc7\x45[\s\S](?P<version>[\s\S]{4})', 
                  pe.get_memory_mapped_image())
version_bytes = match.group("version")
version = ".".join([str(b) for b in version_bytes[::-1]])
print(version)

1.2.7.0


## Decrypt BaseConfig

Pseudocode:

    _BYTE *__usercall Core::getBaseConfig@<eax>(void *base_config@<eax>)
    {
      _BYTE *result; // eax
      int v2; // edx
      int v3; // esi
      int v4; // ecx

      Mem::_copy(base_config, dword_4018C8, 952u);
      v3 = v2;
      v4 = dword_423F88 + dword_423ADC - result;
      do
      {
        *result ^= result[v4];
        ++result;
        --v3;
      }
      while ( v3 );
      return result;
    }

Disassembly:

    .text:00411A23                               Core__getBaseConfig proc near
    .text:00411A23
    .text:00411A23 56                                            push    esi
    .text:00411A24 BA B8 03 00 00                                mov     edx, 3B8h
    .text:00411A29 52                                            push    edx
    .text:00411A2A 68 C8 18 40 00                                push    offset dword_4018C8
    .text:00411A2F 50                                            push    eax
    .text:00411A30 E8 05 2A 00 00                                call    Mem___copy
    .text:00411A35 8B 0D DC 3A 42 00                             mov     ecx, dword_423ADC
    .text:00411A3B 03 0D 88 3F 42 00                             add     ecx, dword_423F88
    .text:00411A41 8B F2                                         mov     esi, edx
    .text:00411A43 2B C8                                         sub     ecx, eax
    .text:00411A45
    .text:00411A45                               loc_411A45:
    .text:00411A45 8A 14 01                                      mov     dl, [ecx+eax]
    .text:00411A48 30 10                                         xor     [eax], dl
    .text:00411A4A 40                                            inc     eax
    .text:00411A4B 4E                                            dec     esi
    .text:00411A4C 75 F7                                         jnz     short loc_411A45
    .text:00411A4E 5E                                            pop     esi
    .text:00411A4F C3                                            retn
    .text:00411A4F                               Core__getBaseConfig endp

In [5]:
match = re.search(rb'\xba(?P<encbuf_len>[\s\S]{2}\x00\x00)\x52\x68(?P<encbuf_addr>[\s\S]{4})',
                   pe.get_memory_mapped_image())

encbuf_len = struct.unpack("I", match.group("encbuf_len"))[0]
encbuf_addr = struct.unpack("I", match.group("encbuf_addr"))[0]
encbuf_rva = encbuf_addr - pe.OPTIONAL_HEADER.ImageBase
encbuf = pe.get_data(encbuf_rva, encbuf_len)

# #define PESECTION_OF_BASECONFIG_KEY 2
section = pe.sections[2]
xor_key = pe.get_data(section.VirtualAddress, encbuf_len)

base_config = b""
for i, eb in enumerate(encbuf):
    pb = eb ^ xor_key[i % len(xor_key)]
    base_config += struct.pack("B", pb)
    
print(base_config)

b'\xb3:\xa7\x88\xdb\xac\xc9\xf6OS\x0e\xba\x8e=\x9awx\\\xe8\xc8\x0f7\xb4\x8e~\xe0\x18\x89[&\xf5&)\x0b\x94\x98!y\x8d*20\xe8\xd4\n\x00\n\x00\xc7D\xb3\xdea}.?\n`)\x9f@\x01\xbd\xbd\x92C\x8f\xbfb\x90\xb0\x82\xe9\xce\xc2/|\x8e]\x0f\r\x86\xb1\xa0\xf4Al\xe8\xf06h\xb4\xd9\\\x0c\x8dl\x1a\xc0\x85\xe0P\xb7(Or3,\x99o m\xb1C\x02~\xcb\x8c\xb6\x83\x86\t\xbb\xdf\xd8\xf4N\xd0!\x907\x05\xb3\x9at\xe8\x7f\x8f\xff\xa2\xbf\x01\x8b\xe4\xef5\x17\x1c\x94w\n\xea\x89\x15\x0f\xf0j\xe1F\x9c\x11?Jz\xcd\xda\xa04\xac\xfa\xf9\xe5\x13i\x041-TZ\xe9u`\x12\x97\xc4\xd7\xf5}b\xaf\x1d\xe3y/\xc3\xc9\xe2\x08\xadd\xee\xd2\xa9\x9b)\xf1g\xf3\x92\xed\'S;=8f\x93\xfd\xabk0\xdc\x0e\xc1\xec\xa7\x8e]\xeb*\xf8B\x16\x8a\xb2"U{\x10\xa5\xdde\xa3\xa6\xe6\xc8\xf6\x87n:s_\xde\x96\xb0[$\x81MA\x88\xcf\xbe\xd3W^Q\xbcL\x9d<aIY\xbd#\xceK\x14\xa1R\xae\xfb\xa4\xd6.V\xb8\xd4\x9f+>|\x19\xfc\x95Xv\xd1HEx\xdb&@\xc5c\x82\xc2q\rD\xaa\x1f\x07\x03\xca\x0b\xc6\xf7%\xba9\xcc\xb5\xfep\xe7\x91\xf2\x98\xd5\x1b2\x18\xa8G\x84\x80\x06\x00\x9e\x1e\xc7\xb9\x00\x00\xfe\

## Parse BaseConfig

In [6]:
flags = struct.unpack("I", base_config[0x220:0x220+4])[0]
print(f"flags: 0x{flags:x}\n")

delayStats = struct.unpack("I", base_config[0x380:0x380+4])[0]
print(f"delayStats: {delayStats}\n")

defaultConfig = base_config[0x2c8:0x2c8+101].split(b"\x00")[0].decode().replace("http", "hxxp")
print(f"defaultConfig: {defaultConfig}\n")

baseKey = base_config[0x59:0x59+258]
print(f"baseKey: {baseKey}\n")
""
defaultBotnet = base_config[0x1c0:0x1c0+21].split(b"\x00\x00")[0].decode()
print(f"defaultBotnet: {defaultBotnet}\n")

delayReport = struct.unpack("I", base_config[0x2c:0x2c+4])[0]
print(f"delayReport: {delayReport}\n")

delayConfig = struct.unpack("I", base_config[0x278:0x278+4])[0]
print(f"delayConfig: {delayConfig}\n")

flags: 0x800

delayStats: 655370

defaultConfig: hxxp://mugspade.ru/orders2010.php

baseKey: b'6h\xb4\xd9\\\x0c\x8dl\x1a\xc0\x85\xe0P\xb7(Or3,\x99o m\xb1C\x02~\xcb\x8c\xb6\x83\x86\t\xbb\xdf\xd8\xf4N\xd0!\x907\x05\xb3\x9at\xe8\x7f\x8f\xff\xa2\xbf\x01\x8b\xe4\xef5\x17\x1c\x94w\n\xea\x89\x15\x0f\xf0j\xe1F\x9c\x11?Jz\xcd\xda\xa04\xac\xfa\xf9\xe5\x13i\x041-TZ\xe9u`\x12\x97\xc4\xd7\xf5}b\xaf\x1d\xe3y/\xc3\xc9\xe2\x08\xadd\xee\xd2\xa9\x9b)\xf1g\xf3\x92\xed\'S;=8f\x93\xfd\xabk0\xdc\x0e\xc1\xec\xa7\x8e]\xeb*\xf8B\x16\x8a\xb2"U{\x10\xa5\xdde\xa3\xa6\xe6\xc8\xf6\x87n:s_\xde\x96\xb0[$\x81MA\x88\xcf\xbe\xd3W^Q\xbcL\x9d<aIY\xbd#\xceK\x14\xa1R\xae\xfb\xa4\xd6.V\xb8\xd4\x9f+>|\x19\xfc\x95Xv\xd1HEx\xdb&@\xc5c\x82\xc2q\rD\xaa\x1f\x07\x03\xca\x0b\xc6\xf7%\xba9\xcc\xb5\xfep\xe7\x91\xf2\x98\xd5\x1b2\x18\xa8G\x84\x80\x06\x00\x9e\x1e\xc7\xb9\x00\x00'

defaultBotnet: u s a

delayReport: 655370

delayConfig: 1966110



## Download DynamicConfig

A PCAP with a DynamicConfig is on [VirusTotal](https://www.virustotal.com/gui/file/db5989f7d5b60c7e22fa5ac38666831929498b3ee1690aec349a6424ba3dfba1).

![](iceix_1.2.7.0_pcap.png)

Pseudocode:

    char __stdcall DynamicConfig::download(char *url)
    {
      ...
      Core::getPeSettings(&pe_settings);
      bot_id = Str::unicodeToXEx(-1, 0, &pe_settings.bot_id);
      Str::unicodeToXEx(-1, 0, &pe_settings.bot_id);
      bot_id_len = Str::_LengthA(bot_id);
      Mem::_copy(rc4_S, &baseConfig.baseKey, 0x102u);
      Mem::copyEx(v4, (bot_id_len + 1));            // copies bot_id to enc_bot_id
      Crypt::_rc4(rc4_S, enc_bot_id, bot_id_len);
      if ( Crypt::_md5Hash(enc_bot_id_hash, enc_bot_id, bot_id_len) )
        md5_hash_to_hex_digest(enc_bot_id_hash_hex, enc_bot_id_hash);
      Mem::_set(post_data, 0, 256u);
      bn1equals_len = Str::_LengthA("bn1=");
      append_like(post_data, v8, bn1equals_len);    // appends "bn1="
      bot_id_len2 = Str::_LengthA(bot_id);
      append_like(post_data, bot_id, bot_id_len2);
      sk1equals_len = Str::_LengthA("&sk1=");
      append_like(post_data, v11, sk1equals_len);   // appends &sk1=
      enc_bot_id_hash_hex_len = Str::_LengthA(enc_bot_id_hash_hex);
      append_like(post_data, v13, enc_bot_id_hash_hex_len);// appends enc_bot_id_hash_hex
      cud.post_data = post_data;
      cud.post_data_len = Str::_LengthA(post_data);
      ...
      
Uses a custom RC4 PRGA function, pseudocode:

    int __userpurge Crypt::_rc4@<eax>(int S@<eax>, int buf, unsigned int buf_len)
    {   
      unsigned int index; // edi 
      char S_i; // dl
      unsigned __int8 j; // [esp+6h] [ebp-2h]
      unsigned __int8 i; // [esp+7h] [ebp-1h]

      i = *(S + 256);
      index = 0;
      for ( j = *(S + 257); index < buf_len; ++index )
      {   
        i += 3;
        S_i = *(i + S); 
        j += S_i + 7;
        *(i + S) = *(j + S); 
        *(j + S) = S_i;
        *(buf + index) ^= *((*(i + S) + S_i) + S); 
      }   
      *(S + 256) = i;
      *(S + 257) = j;
      return S;
    }   

In [7]:
def rc4_keystate(S, in_buf):
    out_buf = b""

    i = S[256]
    j = S[257]
    
    for in_byte in in_buf:
        (i, j, S, K) = rc4_prga(i, j, S)
        out_byte = in_byte ^ K
        out_buf += struct.pack("B", out_byte)

    return out_buf


def rc4_prga(i, j, S): 
    # custom prga
    i = (i + 3) % 256 
    j = (j + S[i] + 7) % 256 

    S = bytearray(S)
    S[i], S[j] = S[j], S[i]
    S = bytes(S)

    K = S[(S[i] + S[j]) % 256]

    return (i, j, S, K)


bot_id = b"WIN7PRO_X86_000_74DEB1E36522DF69_26"
enc_bot_id = rc4_keystate(baseKey, bot_id)

md5 = hashlib.md5()
md5.update(enc_bot_id)
sk1 = md5.hexdigest().upper()

assert(sk1 == "C15CAF65F6280F4916AB79B669689A92")
print(f"{sk1=}")

sk1='C15CAF65F6280F4916AB79B669689A92'


## Decrypt DynamicConfig

Pseudocode:

    ...
    Mem::_copy(v7, rc4Key, 0x102u);
    Crypt::_rc4(v4, binStorage, dataSize);
    if ( dataSize )                             // inlined Crypt::_visualDecrypt
    {
      v5 = dataSize - 1;
      if ( dataSize != 1 )
      {
        do
        {
          *(v5 + binStorage) ^= *(v5 + binStorage - 1);
          --v5;
        }
        while ( v5 );
      }
    }

In [8]:
def visual_decrypt(enc_buf):
    buf = bytearray(enc_buf)

    for i in range(len(buf)-1, 0, -1):
        buf[i] ^= buf[i-1]

    plain_buf = bytes(buf)

    return plain_buf


# Load encrypted DynamicConfig from PCAP
fp = open("setusating.bin", "rb")
config_bin = fp.read()
fp.close()

round1 = rc4_keystate(baseKey, config_bin)
binstorage = visual_decrypt(round1)
print(binstorage)

b'\xe6\xb8\xf8\xe4W\xa1\x91\xdd+\x1b\xfeAl>!R\xb3\xdb\x07\xf0\xcbi\x00\x00\x00\x00\x00\x00/\x00\x00\x00\xb99\x8d\xf6tL^\xa4\xc3\xb2\xc2\xdc\x18\x94\xbb\x05!N\x00\x00\x00\x00\x00\x10\x04\x00\x00\x00\x04\x00\x00\x00\x00\x07\x02\x01"N\x00\x00\x00\x00\x00\x10\x19\x00\x00\x00\x19\x00\x00\x00http://datecoin.ru/us.php#N\x00\x00\x00\x00\x00\x10\x1d\x00\x00\x00\x1d\x00\x00\x00http://trapbath.ru/busted.php$N\x00\x00\x01\x00\x00\x10\xa3\x00\x00\x00\xca\x00\x00\x00\xff\xb7\xff\xffhttp://franksinat\x0842.com/tur\xb5\x0f\xf6\xffkishair.php\x00(clot\r@n\xb7\xbf,.ru/mu1ke!wk\xdb\xdf\xfe\xfeoliveOcesG.net/b\rt\xee>\xb0oJlavo(tax!ell{\xff\x81\xfdKup\x1amoodgum\x18o\xcd\xfd\x07vrd\x1a209.135\x03\xa6\x00\x18\xdb\xda62\xb8\xb3e\x1e\x00\x00\x00\x00\x00\x00\x00\x00$\xff%N\x00\x00\x00\x00\x00\x10/\x00\x00\x00/\x00\x00\x00Nhttp://*\x00S*/login.osmp.ru/*\x00S*/atl.osmp.ru/*\x00\x00\x01\x00\x00\x00\x01\x00\x00@\x82\x01\x00\x00^\x02\x00\x009\xfb\xfe\xcb\x0b\x00<TITLE>\x0c\n/\x0b\x13\xf9\xb1\xff\xbfWelcome to HSBC

## Parse DynamicConfig

In [9]:
def parse_binstorage_header(binstorage):
    header = {
        "randData": binstorage[0:20],
        "size": struct.unpack("I", binstorage[20:24])[0],
        "flags": struct.unpack("I", binstorage[24:28])[0],
        "count": struct.unpack("I", binstorage[28:32])[0],
        "md5Hash": binstorage[32:48],
    }
        
    return header


def parse_binstorage_sections(section_data, num_sections):
    CFGIDS = {
        20001: "CFGID_LAST_VERSION",
        20002: "CFGID_LAST_VERSION_URL",
        20003: "CFGID_URL_SERVER_0",
        20004: "CFGID_URL_ADV_SERVERS",
        20005: "CFGID_HTTP_FILTER",
        20006: "CFGID_HTTP_POSTDATA_FILTER",
        20007: "CFGID_HTTP_INJECTS_LIST",
        20008: "CFGID_DNS_LIST",
    }
    
    sections = []
    offset = 0
    for i in range(num_sections):
        section = {}
        
        id_ = struct.unpack("I", section_data[offset:offset+4])[0]
        if int(id_) in CFGIDS:
            section["id"] = f'{CFGIDS[int(id_)]} ({id_})'
        else:
            section["id"] = id_
            
        section["flags"] = struct.unpack("I", section_data[offset+4:offset+8])[0]
        section["size"] = struct.unpack("I", section_data[offset+8:offset+12])[0]
        section["realSize"] = struct.unpack("I", section_data[offset+12:offset+16])[0]
        # additional parsing of data is left as an exercise
        section["data"] = section_data[offset+16:offset+16+section["size"]]
        
        offset += 4 + 4 + 4 + 4 + section["size"]
        
        sections.append(section)
        
    return sections


binstorage_header = parse_binstorage_header(binstorage)
print("binstorage_header:\n")
pprint.pprint(binstorage_header)

binstorage_sections = parse_binstorage_sections(binstorage[48:], binstorage_header["count"])
print()
print("binstorage_sections:\n")
pprint.pprint(binstorage_sections)

binstorage_header:

{'count': 47,
 'flags': 0,
 'md5Hash': b'\xb99\x8d\xf6tL^\xa4\xc3\xb2\xc2\xdc\x18\x94\xbb\x05',
 'randData': b'\xe6\xb8\xf8\xe4W\xa1\x91\xdd+\x1b\xfeAl>!R\xb3\xdb\x07\xf0',
 'size': 27083}

binstorage_sections:

[{'data': b'\x00\x07\x02\x01',
  'flags': 268435456,
  'id': 'CFGID_LAST_VERSION (20001)',
  'realSize': 4,
  'size': 4},
 {'data': b'http://datecoin.ru/us.php',
  'flags': 268435456,
  'id': 'CFGID_LAST_VERSION_URL (20002)',
  'realSize': 25,
  'size': 25},
 {'data': b'http://trapbath.ru/busted.php',
  'flags': 268435456,
  'id': 'CFGID_URL_SERVER_0 (20003)',
  'realSize': 29,
  'size': 29},
 {'data': b'\xff\xb7\xff\xffhttp://franksinat\x0842.com/tur\xb5\x0f\xf6\xffkish'
          b'air.php\x00(clot\r@n\xb7\xbf,.ru/mu1ke!wk\xdb\xdf\xfe\xfeoliveOces'
          b'G.net/b\rt\xee>\xb0oJlavo(tax!ell{\xff\x81\xfdKup\x1amoodgu'
          b'm\x18o\xcd\xfd\x07vrd\x1a209.135\x03\xa6\x00\x18\xdb\xda62\xb8\xb3e'
          b'\x1e\x00\x00\x00\x00\x00\x00\x00\x00$\xff',
 

 {'data': b'\xdf\xfdw\xcb\x04\x00\x0b\x03</head>\x88\x03\nstyle\xff'
          b'\xbf\xbf\xfd id="h\x04body">\x05 { display\xd6\xfe\x7f\xfb: non!!im'
          b'portant; }@6\xb7w\xbf=>\r\n?cript\tsetT#eo\xdb\xda\xb7\xffut(func'
          b'ti5 ( )H\x1a \x00ko\x7f\xb7trSvar s =Zocu*J\xd8\xff\xdav.g4EG\nByI'
          b"d('\x83>\xbb\xbd=');5\tif@s)>s.p\xed\xb7[\xd7;&N\x1de.\x08mov"
          b'eC,\xfd\xf6\xff\xddld ;} catch(e){}-}, 10\x16,\xd9\xcf\x00B<'
          b'/\x9e\xa8\x87=k\xff\xf6ype\xebtext/ja\x8f\x15" s\x01s\xbf\xe8ngua'
          b"g\x1a\x156\xb1\xdb\xfd\xad\xfdMyCo\xfc\xd7erHASH\xbe'%\xa4\xfc\r{"
          b'{tna\xb6%\';"\x1asub.\xd6\x92-4r \xdb\x1f\xf9\xef\x86\xdb\r\x03'
          b'win\xf5w.Jrro#\xdb\xde~\x08$(msg\xb2url\x04l#\xdb`\xadpe_m\xc6,'
          b"c\x1dsd\x1f\xec\x86o\x9d&& \n.log\x0e\x05{l7(:+';:'+B\xb0"
          b"\xe0\\\xcc\x10g\x11^\n \xe1p\xc3\xb62t'n\x85ue;$2\x85U\t\x01'"
          b'\x9d\x16\xb6\x7f\xc7rc?ttps://a\x19x\xd8oc]\xe3\xd6\x99\x9fap'
         

## Commands

Pseudocode:

    ...
    ci = 0;
    while ( 1 )
    {
      CryptedStrings::_getW(word_4017F8[4 * ci], commandNameBuffer);
      if ( !lstrcmpiW(*args, commandNameBuffer) )
        break;
      if ( ++ci >= 26 )
        goto LABEL_33;
    ...
    
Disassembly:

    .text:00414144 33 FF                                         xor     edi, edi
    .text:00414146
    .text:00414146                               loc_414146:
    .text:00414146 66 8B 04 FD F8 17 40 00                       mov     ax, ds:word_4017F8[edi*8]
    .text:0041414E 8D B5 54 FD FF FF                             lea     esi, [ebp+commandNameBuffer]
    .text:00414154 E8 C4 CD FF FF                                call    CryptedStrings___getW
    .text:00414159 8B C6                                         mov     eax, esi
    .text:0041415B 50                                            push    eax
    .text:0041415C FF 33                                         push    dword ptr [ebx]
    .text:0041415E FF 15 88 12 40 00                             call    ds:lstrcmpiW
    .text:00414164 85 C0                                         test    eax, eax
    .text:00414166 74 08                                         jz      short loc_414170
    .text:00414168 47                                            inc     edi
    .text:00414169 83 FF 1A                                      cmp     edi, 1Ah
    .text:0041416C 72 D8                                         jb      short loc_414146
    .text:0041416E EB 1F                                         jmp     short loc_41418F

In [10]:
match = re.search(rb'\x66\x8b\x04\xfd(?P<indexes_addr>[\s\S]{4})', pe.get_memory_mapped_image())
indexes_addr = struct.unpack("I", match.group("indexes_addr"))[0]

for i in range(26):
    index_addr = indexes_addr + i * 8
    index_rva = index_addr - pe.OPTIONAL_HEADER.ImageBase
    enc_string_index = pe.get_word_at_rva(index_rva)
    
    plain_string = decrypt_string(pe, enc_string_index)
    print(f"{enc_string_index}: {plain_string}")

164: os_shutdown
165: os_reboot
166: bot_uninstall
167: bot_update
168: bot_update_exe
169: bot_bc_add
170: bot_bc_remove
171: bot_httpinject_disable
172: bot_httpinject_enable
173: fs_path_get
174: fs_search_add
175: fs_search_remove
176: user_destroy
177: user_logoff
178: user_execute
179: user_cookies_get
180: user_cookies_remove
181: user_certs_get
182: user_certs_remove
183: user_url_block
184: user_url_unblock
185: user_homepage_set
186: user_ftpclients_get
187: user_emailclients_get
188: user_flashplayer_get
189: user_flashplayer_remove
